<b> A matroid $M$ is said to be 1-GP if, for every nonmodular pair of flats $(F_1,F_2)$ of $M$, there is a proper point extension $M'$ of $M$ by an element $p$ such that $p$ lies in the intersection of the closures of $F_1$ and $F_2$ in $M'$. Observe that by proper point extension, we mean a non-loop extension of $M$. In other words, rank of $p$ is 1.</b>

<b>A matroid $M$ is said to be $k$-GP, for some integer $k\geq 2$ if, for every nonmodular pair of flats, there exists a sequence of $k$ matroids $M=M_0,M_1,\ldots,M_{k-1}$ such that each matroid $M_i$ in the sequence is a proper point extension of its predecessor (determined by a modular cut containing the pair of flats being checked at each turn) and is a ($k-i$)-GP matroid.</b>

In [1]:
import numbers
from functools import lru_cache

In [2]:
def CheckModular(M,F1,F2):
    '''
    Takes as input the matroid and the flats F1 and F2.
    Returns boolean indicating if they form a modular pair.
    '''

    rF1 = M.rank(F1)
    rF2 = M.rank(F2)
    rF12 = M.rank(F1.union(F2))
    rF1n2 = M.rank(F1.intersection(F2))
    if rF1 + rF2 == rF12 + rF1n2:
        return True
    return False

def GetFlats(M):
    '''
    Obtains all hyperplanes and lines of the given matroid.
    '''

    hyps = list()
    for i in range(2,M.rank()):
        fl = [flat for flat in M.flats(i)]
        hyps.extend(fl)

    return hyps

def GetExtensions(M,f1,f2):
    '''
    Given a matroid and a nonmodular pair of flats,
    it returns all the extensions of the matroid in which 
    the corresponding modular cuts contain this pair.
    Outputs:   All extensions corresponding to f1 and f2.
              i -- The new element.
    '''

    G = M.groundset()

    i = -1
    
    while i in G:
        i -= 1
    return M.extensions(i,subsets=[f1,f2]), i

def CheckGP(M):
    '''
    Takes as input a matroid and checks if it is 1-GP.
    If it is not 1-GP, the function returns the boolean
    False, and the pair that breaks 1-GP.
    '''

    flats = GetFlats(M)

    for i in range(len(flats)):
        f1 = flats[i]
        for j in range(i+1,len(flats)):
            f2 = flats[j]
            if CheckModular(M,f1,f2):
                continue
            if M.closure(set()) in M.modular_cut([f1,f2]):   
                return False, f1, f2
    return True, []

@lru_cache(maxsize=256)
def RecursiveCheckGP(M,depth):
    '''
    The recursive function.
    Takes as input the matroid to be checked
    and the depth at which GP is to be checked.
    Returns boolean indicating if the matroid is 
    GP at given depth.
    The print statements can be surpressed.

    input: M (matroid) sage Matroid object
           depth (int) greater than 0
    output: Boolean
    '''
    
    if not isinstance(depth,numbers.Integral) or depth <= 0:
        print('###### Depth has to be a positive integer ######')
        return None
    
    if depth == 1:
        ### We always check GP at depth 1. ###
        res = CheckGP(M) 
        if res[0]:
            return True
        else:
            #print('# nonmodular pair is {} and {} at depth 1'.format(res[1],res[2]))
            return False
    
    ### We start by generating all flats of the matroid. ###
    flats = GetFlats(M)
    
    for k in range(len(flats)):
        f1 = flats[k]
        for l in range(k+1,len(flats)):
            f2 = flats[l]
            
            if CheckModular(M,f1,f2):
                ### We check modularity for each pair of flats. ###
                continue    ### We only want nonmodular pairs. ###

            ### For a nonmodular pair, we generate all extensions corresponding to them. ###
            GPE, i = GetExtensions(M,f1,f2)
            
            GPmatroid = False  ### The indicator we use when we find an appropriate GP extension. ###
            for N in GPE:  

                #nonmodflats = [f1,f2]  
                if RecursiveCheckGP(N,depth-1): 
                    ### Recursion takes place here. ###
                    
                    if N.rank([i]) != 0:
                        ### The moment we find a proper point extension that is GP, we break for that depth. ###
                        GPmatroid = True
                        break
            
            if not GPmatroid:
                #print('# nonmodular pair is {} at depth {}'.format(nonmodflats,depth))
                return False
    return True


In [3]:
Mvam = Matroid(bases=['0124','0126','0125','0127','0234','0246','0245','0247','0236','0235','0237','0256','0267','0257',
'0134','0146','0147','0136','0135','0137','0156','0157','0346','0345','0347','0456','0467','0457',
'0356','0367','0357','0567','1234','1246','1245','1247','1236','1235','1237','1256','1267','1257',
'2346','2347','2456','2467','2457','2356','2367','2357','2567','1346','1345','1347','1456','1467',
'1457','1356','1367','1357','1567','3456','3467','3457','3567'])

MP1 = Matroid(bases=['0123', '0124', '0125', '0126', '0134', '0135', '0137', '0145', '0146', '0147', '0156', '0157', '0167', '0234', '0236', '0237', '0245', '0246', '0247', '0256', '0257', '0267', '0345', '0346', '0356', '0357', '0367', '0457', '0467', '0567', '1235', '1236', '1237', '1245', '1246', '1247', '1257', '1267', '1345', '1346', '1347', '1356', '1357', '1367', '1456', '1467', '1567', '2345', '2346', '2347', '2356', '2357', '2367', '2456', '2457', '2567', '3456', '3457', '3467', '3567', '4567'])

cgp = RecursiveCheckGP(MP1,1)
print(cgp)

True
